In [3]:
import os
import json
import pandas as pd
from tqdm import tqdm
import numpy as np

In [4]:
df = pd.read_csv('train_ver2.csv')

/var/folders/bn/rrr4fsw14591zqtjsxz419z40000gn/T/ipykernel_93102/1621394854.py:1: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('train_ver2.csv')


In [5]:
# Loại bỏ những feature không cần thiết + lọc dữ liệu user của tháng 6/2015
def preprocess(df):
    df = df[df['fecha_dato'] == '2015-06-28']
    df = df.drop(columns=['fecha_dato', 'fecha_alta', 'ult_fec_cli_1t', 'tipodom', 'cod_prov'])
    
    # Convert the columns to numeric
    df[df.columns[-24:]] = df[df.columns[-24:]].apply(pd.to_numeric, errors='coerce')
    
    # Filter rows where the sum of the last 18 columns is greater than 0
    new_df = df.loc[df.iloc[:, -24:].sum(axis=1) > 0]
    
    return new_df

# Kết hợp dữ liệu user tháng 6/2015 và dữ liệu mua của tháng 5/2015
def lag_1_product(df):
    new_df = preprocess(df)
    user_ids = new_df['ncodpers'].unique()
    may_user_values = df[(df['ncodpers'].isin(user_ids)) & (df['fecha_dato'] == '2015-05-28')]

    shifted_1_value_product_per_user = []
    for user_id in tqdm(user_ids):
        user_values = may_user_values[may_user_values['ncodpers'] == user_id]
        last_24_values = user_values.iloc[:, -24:].values.flatten().tolist()
        shifted_1_value_product_per_user.append(last_24_values)
    new_df['shifted_1'] = shifted_1_value_product_per_user
    return new_df

train_df = lag_1_product(df)
train_df = train_df[train_df['shifted_1'].apply(len) == 24]

100%|██████████| 616023/616023 [02:55<00:00, 3505.06it/s]


In [6]:
# Chuẩn hóa dữ liệu cho từng feature
def assert_dtype(df, dtype='test'):
    df.fillna(method='ffill', inplace=True)
    df = df[df['age']!=' NA']
    df['antiguedad'] = df['antiguedad'].apply(lambda x: pd.to_numeric(str(x).replace('[^\d.-]', ''), errors='coerce'))

    if dtype=='test':
        df = df.astype(dtype={'ind_actividad_cliente':"float", 'age': 'int',"indrel":"float", "ind_nuevo":"float" })
        return df
                                                                            
    df = df.astype(dtype={"ind_ahor_fin_ult1":"int8",'ind_aval_fin_ult1':"int8", 'ind_cco_fin_ult1':"int8", 
                                                                      'ind_cder_fin_ult1':"int8", 'ind_cno_fin_ult1':"int8", 'ind_ctju_fin_ult1':"int8", 'ind_ctma_fin_ult1':"int8", 
                                                                      'ind_ctop_fin_ult1':"int8", 'ind_ctpp_fin_ult1':"int8",
                                                                      'ind_nomina_ult1':"int8", 'ind_nom_pens_ult1':"int8",
                                                                      'ind_actividad_cliente':"float", 'age': 'int',
                                                                      "indrel":"float", "ind_nuevo":"float",
                                                                      'ind_deco_fin_ult1':"int8", 'ind_deme_fin_ult1':"int8", 'ind_dela_fin_ult1':"int8",
                                                                      'ind_ecue_fin_ult1':"int8", 'ind_fond_fin_ult1':"int8", 'ind_hip_fin_ult1':"int8",
                                                                      'ind_plan_fin_ult1':"int8", 'ind_pres_fin_ult1':"int8", 'ind_reca_fin_ult1':"int8", 'ind_tjcr_fin_ult1':"int8", 
                                                                      'ind_valo_fin_ult1':"int8", 'ind_viv_fin_ult1':"int8", 'ind_recibo_ult1':"int8"})
    return df

train_df = assert_dtype(train_df, dtype='train')

/var/folders/bn/rrr4fsw14591zqtjsxz419z40000gn/T/ipykernel_93102/3416394411.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
/var/folders/bn/rrr4fsw14591zqtjsxz419z40000gn/T/ipykernel_93102/3416394411.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['antiguedad'] = df['antiguedad'].apply(lambda x: pd.to_numeric(str(x).replace('[^\d.-]', ''), errors='coerce'))


In [7]:
categorical_col = ['ind_empleado', 'pais_residencia', 'sexo', 'ind_nuevo', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall', 'nomprov', 'ind_actividad_cliente', 'segmento']
interval_col = ['antiguedad', 'age', 'renta']
#Label col == các feature sẽ mua ở tháng 6
label_col = train_df.columns[-25:-1]

In [8]:
# Tạo các dict transform các feature
list_encoded = {}
for col in categorical_col:
    mapping_value = {}
    categorical = list(df[col].unique())
    for id, cate in enumerate(categorical):
        mapping_value[cate] = id
    list_encoded[col] = mapping_value

In [9]:
#Các function transform/encode các feature
def transform_age(age):
    if isinstance(age, str):
        return 0
    if age < 20:
        return 0
    elif age < 30:
        return 1
    elif age < 40:
        return 2
    elif age < 50:
        return 3
    elif age < 60:
        return 4
    else:
        return 5


def transform_categorical(dict_, x):
    if x not in dict_.keys():
        return -1
    return dict_[x]

def transform_membership(month):
    if isinstance(month, str):
        return 0
    if month < 24:
        return 0
    elif month < 60:
        return 1
    elif month < 120:
        return 2
    else: 
        return 3

def transform_income(income):
    if isinstance(income, str):
        return 0
    if income < 5e4:
        return 0
    elif income < 1e5:
        return 1
    elif income < 5e5:
        return 2
    else:
        return 3

def transform_value(df):
    for col in tqdm(categorical_col):
        df[col] = df[col].apply(lambda x: transform_categorical(list_encoded[col], x))
    df['age'] = df['age'].apply(lambda x: transform_age(x))
    df['antiguedad'] = df['antiguedad'].apply(lambda x: transform_membership(x))
    df['renta'] = df['renta'].apply(lambda x: transform_income(x)) 
    return df

train = transform_value(train_df)

100%|██████████| 13/13 [00:02<00:00,  6.22it/s]


In [10]:
# Phần xử lý dữ liệu tập test tương tự như tập Train
test = pd.read_csv('test_ver2.csv')

def preprocess(df):
    df = df.drop(columns=['fecha_dato', 'fecha_alta', 'ult_fec_cli_1t', 'tipodom', 'cod_prov'])
    return df

def lag_1_product(df, train_df):
    new_df = preprocess(df)
    user_ids = new_df['ncodpers'].unique()
    may_user_values = train_df[(train_df['ncodpers'].isin(user_ids)) & (train_df['fecha_dato'] == '2016-05-28')]

    shifted_1_value_product_per_user = []
    for user_id in tqdm(user_ids):
        user_values = may_user_values[may_user_values['ncodpers'] == user_id]
        last_24_values = user_values.iloc[:, -24:].values.flatten().tolist()
        shifted_1_value_product_per_user.append(last_24_values)
    new_df['shifted_1'] = shifted_1_value_product_per_user
    return new_df

test_lag_1 = lag_1_product(test, df)

/var/folders/bn/rrr4fsw14591zqtjsxz419z40000gn/T/ipykernel_93102/4132674413.py:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv('test_ver2.csv')
100%|██████████| 929615/929615 [06:10<00:00, 2507.39it/s]


In [11]:
# Concat các feature lại với nhau, mỗi sample training = 16 feature user information + 24 pass value purchase
def generate_dataset(df):
    X_1 = df[categorical_col].values
    X_2 = df[interval_col].values
    X_3 = np.array(df['shifted_1'].tolist())
    X = np.concatenate((X_1, X_2, X_3), axis=-1)
    if label_col[0] not in df.columns:
        return X
    y = df[label_col].values    
    assert X.shape[0] == y.shape[0]
    return X, y

Vì ta sẽ đưa dữ liệu vào một model dạng phân loại, mỗi feature training X sẽ dự đoán 1 sản phẩm tiếp theo, tuy nhiên theo dữ liệu ta có mỗi tháng 1 user có thể mua nhiều hơn 1 sản phẩm. Giả sử, 1 tháng A mua n sản phẩm => tách y thành n sample training, mỗi sample chỉ chứa 1 sản phẩm.

In [12]:
import random
def remap_data(X, y):
    new_x, new_y = [], []
    for id, sample in tqdm(enumerate(y)):
        if sum(sample) > 0:
            indices = random.choice(np.where(np.array(sample) == 1)[0])
            new_x.append(X[id])
            new_y.append([indices])
    assert len(new_x) == len(new_y)
    return np.array(new_x), np.array(new_y)

In [13]:
X, y = generate_dataset(train)
X, y = remap_data(X, y)
X_filled = np.nan_to_num(X, nan=-1) #Fill nan value
X_test = generate_dataset(transform_value(test_lag_1))

610875it [00:04, 125377.75it/s]
100%|██████████| 13/13 [00:02<00:00,  4.39it/s]


In [19]:
# Hàm xử lý dữ liệu inference sinh ra submit file
mapping_product = {}
for idx, product in enumerate(df.columns[-24:]):
    mapping_product[idx] = product
def get_submit_file(test_result):
    test_result = test_result.tolist()
    df = pd.DataFrame(zip(test['ncodpers'].tolist(), test_result), columns=['ncodpers', 'added_products'])
    df['added_products'] = df['added_products'].apply(lambda x: mapping_product[x])
    with open('submission.csv', 'w') as f:
        f.write('ncodpers,added_products')
        f.write('\n')
        for id in tqdm(range(len(df))):
            item = df.loc[id]
            f.write(f"""{item['ncodpers']},{item['added_products']}""")
            f.write('\n')
        f.close()

In [36]:
from sklearn.model_selection import train_test_split

X_train, X_dev, y_train, y_dev = train_test_split(X_filled, y ,test_size=0.3, random_state=42, stratify=y)

### XGBoost

In [32]:
max_depth=[4,5,6]
number_tree=[10,50,100]
max_delta_step=[0.1,0.5,1]

In [33]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
record, model_list = [], []
for depth in max_depth:
    for tree in number_tree:
        for step in max_delta_step:
            bst = XGBClassifier(n_estimators=tree, 
                                max_depth=depth, 
                                max_delta_step=step, 
                                objective='multi:softprob')
            bst.fit(X=X_train, y=y_train)
            prediction_train = bst.predict(X_train)
            train_acc = accuracy_score(y_train, prediction_train)
            prediction = bst.predict(X_dev)
            dev_acc = accuracy_score(y_dev, prediction)
            model_list.append(bst)
            record.append([depth, tree, step, train_acc, dev_acc])

In [34]:
result = pd.DataFrame(record, columns=['max_depth', 'number_tree', 'max_step', 'train_acc', 'dev_acc'])

In [35]:
result

,max_depth,number_tree,max_step,train_acc,dev_acc
0,4,10,0.1,0.749799,0.749982
1,4,10,0.5,0.763161,0.761370
2,4,10,1.0,0.763405,0.760235
3,4,50,0.1,0.760409,0.759586
4,4,50,0.5,0.772350,0.761774
5,4,50,1.0,0.773138,0.761054
6,4,100,0.1,0.769716,0.761185
7,4,100,0.5,0.780032,0.761567
8,4,100,1.0,0.780663,0.761305
9,5,10,0.1,0.752804,0.753038


### Logistic Regression

In [37]:
X_train, X_dev, y_train, y_dev = train_test_split(X_filled, y ,test_size=0.1, random_state=42, stratify=y)

In [61]:
y_train.shape

(549787, 1)

In [38]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_train, y_train)

/opt/homebrew/Caskroom/miniforge/base/envs/rcm/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/homebrew/Caskroom/miniforge/base/envs/rcm/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [39]:
y_pred = clf.predict(X_dev)

In [58]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import OneHotEncoder

def eval_logistic(y_pred, y_truth):
    record = []
    encoder = OneHotEncoder()
    y_truth = encoder.fit_transform(y_truth).toarray()
    y_pred = encoder.transform(np.reshape(y_pred, (y_pred.shape[0], 1))).toarray()

    for class_ in range(y_pred.shape[1]):
        label = label_col[class_]
        error = mse(y_truth[:, class_], y_pred[:, class_])
        record.append([label, error])
    return record

df_eval = pd.DataFrame(eval_logistic(y_pred, y_dev), columns=['product', 'MSE'])

In [59]:
df_eval

,product,MSE
0,ind_ahor_fin_ult1,0.000049
1,ind_aval_fin_ult1,0.129436
2,ind_cco_fin_ult1,0.000196
3,ind_cder_fin_ult1,0.049355
4,ind_cno_fin_ult1,0.000360
5,ind_ctju_fin_ult1,0.004764
6,ind_ctma_fin_ult1,0.059111
7,ind_ctop_fin_ult1,0.019791
8,ind_ctpp_fin_ult1,0.001522
9,ind_deco_fin_ult1,0.000737


In [60]:
y_test = clf.predict(X_test)
get_submit_file(y_test)

100%|██████████| 929615/929615 [00:17<00:00, 52852.66it/s]
